## Data Acquisition

Install the following necessary packages

In [1]:

#pip install pandas
#pip install numpy
#pip install scipy
#pip install yahoo_finance_api2
#pip install pandas-datareader
#pip install ta
#pip install seaborn
#pip install sklearn
#conda install tensorflow


In [2]:
import pandas as pd
import numpy as np

# Importing tickers/Symbols field from the scrapped stockdata

df = pd.read_csv('/home/nielit/Desktop/datasample.csv',header=None)
df.columns=["ticker"]
df.head()

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


,ticker
0,RELIANCE.NS
1,RELIANCE.BO
2,TCS.NS
3,TCS.BO
4,HDFCBANK.NS


In [3]:
len(df.ticker)

1981

In [4]:
len(df.ticker.unique())

1980

In [5]:
# Check for duplicate values

df_dup = df[df.duplicated()]
df_dup

,ticker
300,JINDALSTEL.BO


In [6]:
# Drop the duplicate values

df = df.drop(df.index[300])

In [7]:
tic = list(df.ticker)

In [8]:
# Import necessary packages

import pandas_datareader as pdr
from datetime import datetime

In [9]:
from tqdm import tqdm

In [10]:
# Collecting the stock market data from open source yahoo finance API for the following stock tickers for past 2 years

data = list()
for i in tqdm(tic):
    try:
        data.append(pdr.get_data_yahoo(symbols=i, start=datetime(2018, 1, 2), end=datetime(2019, 12, 31)))
    except:
        pass

100%|██████████| 1980/1980 [57:10<00:00,  1.24s/it] 


In [11]:
for i in range(len(data)):
    
    data[i]['Symbol'] = tic[i]

In [12]:
df = pd.DataFrame(data[0])

for i in np.arange(1,len(data)):
    df = df.append(data[i])

In [13]:
df.isnull().sum()

High         0
Low          0
Open         0
Close        0
Volume       0
Adj Close    0
Symbol       0
dtype: int64

Our required dataframe df looks something like this..

In [14]:
df.head()

,High,Low,Open,Close,Volume,Adj Close,Symbol
Date,,,,,,,
2018-01-02,919.549988,906.400024,913.000000,911.150024,4342815.0,900.578369,RELIANCE.NS
2018-01-03,926.000000,913.049988,925.000000,914.799988,6175312.0,904.185974,RELIANCE.NS
2018-01-04,921.799988,915.700012,918.150024,920.299988,4118581.0,909.622131,RELIANCE.NS
2018-01-05,926.900024,920.250000,921.799988,923.250000,3401905.0,912.537903,RELIANCE.NS
2018-01-08,931.000000,923.500000,926.099976,928.549988,4035417.0,917.776428,RELIANCE.NS


In [15]:
# Saving the dataframe in local

df.to_csv("/home/nielit/Desktop/StockData2.csv")

In [16]:
# Loading the dataframe back from local 

df_stock = pd.read_csv("/home/nielit/Desktop/StockData2.csv")

In [22]:
# The stock data is in the exact format to store in a database

df_stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 926833 entries, 0 to 926832
Data columns (total 8 columns):
Date         926833 non-null object
High         926833 non-null float64
Low          926833 non-null float64
Open         926833 non-null float64
Close        926833 non-null float64
Volume       926833 non-null float64
Adj Close    926833 non-null float64
Symbol       926833 non-null object
dtypes: float64(6), object(2)
memory usage: 56.6+ MB


# Using MongoDB as my database

In [1]:
from pymongo import MongoClient 
from random import randint
try: 
    client_mongo  = MongoClient() 
    print("Connected successfully!!!") 
except:   
    print("Could not connect to MongoDB") 

Connected successfully!!!


In [6]:
# Creating a database db and collection stock

db = client_mongo.db_6
collection = db.stock

### Importing the dataset into database using MongoClient

In [25]:
for i in df_stock.values:
    collection.insert_one({"Date":i[0],"High":i[1],"Low":i[2],"Open":i[3],"Close":i[4],"Volume":i[5],"Adj Close":i[6],"Symbol":i[7]})

The stock data is now stored in MongoDB

### Exporting the dataset from MongoClient back here

In [7]:
df = pd.DataFrame(list(db.stock.find()))
df.head()

,Adj Close,Close,Date,High,Low,Open,Symbol,Volume,_id
0,900.578369,911.150024,2018-01-02,919.549988,906.400024,913.000000,RELIANCE.NS,4342815.0,5e1cc8bb2e3f9313fcac3063
1,904.185974,914.799988,2018-01-03,926.000000,913.049988,925.000000,RELIANCE.NS,6175312.0,5e1cc8bb2e3f9313fcac3064
2,909.622131,920.299988,2018-01-04,921.799988,915.700012,918.150024,RELIANCE.NS,4118581.0,5e1cc8bb2e3f9313fcac3065
3,912.537903,923.250000,2018-01-05,926.900024,920.250000,921.799988,RELIANCE.NS,3401905.0,5e1cc8bb2e3f9313fcac3066
4,917.776428,928.549988,2018-01-08,931.000000,923.500000,926.099976,RELIANCE.NS,4035417.0,5e1cc8bb2e3f9313fcac3067


In [8]:
# Reformatting the data back into our desired form from MongoDB

df = df.drop(['_id'],axis=1)
df = df[['Date','High','Low','Open','Close','Volume','Adj Close','Symbol']]
df = df.set_index('Date')
df.head()

,High,Low,Open,Close,Volume,Adj Close,Symbol
Date,,,,,,,
2018-01-02,919.549988,906.400024,913.000000,911.150024,4342815.0,900.578369,RELIANCE.NS
2018-01-03,926.000000,913.049988,925.000000,914.799988,6175312.0,904.185974,RELIANCE.NS
2018-01-04,921.799988,915.700012,918.150024,920.299988,4118581.0,909.622131,RELIANCE.NS
2018-01-05,926.900024,920.250000,921.799988,923.250000,3401905.0,912.537903,RELIANCE.NS
2018-01-08,931.000000,923.500000,926.099976,928.549988,4035417.0,917.776428,RELIANCE.NS


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 926833 entries, 2018-01-02 to 2020-01-01
Data columns (total 7 columns):
High         926833 non-null float64
Low          926833 non-null float64
Open         926833 non-null float64
Close        926833 non-null float64
Volume       926833 non-null float64
Adj Close    926833 non-null float64
Symbol       926833 non-null object
dtypes: float64(6), object(1)
memory usage: 56.6+ MB
